# Transform

**Transfom** represents one or more transform matrices, like translation matrices, scaling matrices, rotation matrices and projection matrices.

## Identity

In [ ]:
from py3d import Transform
Transform()


In [ ]:
assert _.sum() == 4


In [ ]:
Transform().tile(2)


In [ ]:
assert _.sum() == 8


## Translation

### Translation Matrix

In **py3d**, transformations are represented as left-multiplication matrixes, point are represented as row vectors.

#### Left-multiplication translation matrix

In [ ]:
from sympy import symbols, Matrix
x, y, z, dx, dy, dz = symbols("x y z dx dy dz")
point = Matrix([x, y, z, 1]).T
translation = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [dx, dy, dz, 1]
])
translation


In [ ]:
point * translation


In [ ]:
dx_,dy_,dz_ = symbols("dx' dy' dz'")
translation_ = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [dx_, dy_, dz_, 1]
])
translation_ * translation

In [ ]:
translation * translation_

#### Right-multiplication translation matrix

In [ ]:
translation.T


In [ ]:
translation.T * point.T


In [ ]:
import py3d
py3d.Transform.from_translation([2, 3, 4])


In [ ]:
import numpy
assert numpy.array_equal(_[3, 0:3], [2, 3, 4])


In [ ]:
import py3d
py3d.Transform.from_translation(x=1).tile(2)


Translate a series of points

In [ ]:
import py3d
points = py3d.Vector3.grid(range(5), range(5)).as_point()
points.color = py3d.Color(g=1)
py3d.render(points)
points @= py3d.Transform.from_translation(x=2, y=1, z=3)
points.color = py3d.Color(r=1)
py3d.render(points)


In [ ]:
import py3d
c=py3d.cube(0.5,0.2,0.3)
c @= py3d.Transform.from_translation(x=range(-10,10))
c.paint()

Move a car

In [ ]:
import py3d

car = py3d.car()
car.color = py3d.Color(r=1)
t = 0
while t < 4:
    py3d.render(car, t=t)
    car.xyz @= py3d.Transform.from_translation(x=0.2)
    t += 0.1
py3d.render(car, t=t)


## Scaling

In [ ]:
import py3d
py3d.Transform.from_scaling(x=1, y=2, z=(1, 2, 3))

In [ ]:
from py3d import Vector3, Transform
points = (Vector3.rand(1000)-0.5).U @ Transform.from_scaling(x=2)
points.as_point()


## [Rotation](Rotation.html)

## Transposition

With property **T**, we can get transposed matrices of a given **Transform**. 
The matrices represented by the **Transform** will be transposed respectively and return a new **Tranform** with the same shape.

In [ ]:
import py3d
transform=py3d.Transform([
           [[ 0.54030231,  0.84147098,  0.        ,  0.        ],
            [-2.01953036,  1.29672553,  0.        ,  0.        ],
            [ 0.        ,  0.        ,  1.        ,  0.        ],
            [ 1.4       ,  0.        ,  0.        ,  1.        ]],

           [[-0.41614684,  0.90929743,  0.        ,  0.        ],
            [-2.81882202, -1.29005519,  0.        ,  0.        ],
            [ 0.        ,  0.        ,  1.        ,  0.        ],
            [ 2.1       ,  0.        ,  0.        ,  1.        ]],

           [[ 0.96017029, -0.2794155 ,  0.        ,  0.        ],
            [ 2.17944089,  7.48932824,  0.        ,  0.        ],
            [ 0.        ,  0.        ,  1.        ,  0.        ],
            [ 6.8       ,  0.        ,  0.        ,  1.        ]]])
transform.T

## Interpolation

In [ ]:
import py3d, numpy
c = py3d.axis(1)
poses = py3d.Vector([
    [0, 5, 1, 0, 2, 3],
    [2, 7, 9, 1, 4, 8],
    [10, 3, 8, 6, 9, 5]
], columns=["x", "y", "z", "rvx", "rvy", "rvz"])
t = py3d.Transform.from_rotation_vector(poses[["rvx","rvy","rvz"]])@py3d.Transform.from_translation(poses[["x","y","z"]])
c @ t.lerp(numpy.linspace(0, 4, 20), [0, 1, 4])

## Decomposition

Decompose a transformation matrix to scaling, rotation and translation.

In [ ]:
t.scaling_vector

In [ ]:
t.translation_vector

In [ ]:
t.rotation_vector

## Projection

### Camera and scene

In [ ]:
import py3d
camera_axis=py3d.axis(2)
camera = py3d.camera(2000, 1000, 3000)
transform = py3d.Transform.from_translation([0,0,5]) @ py3d.Transform.from_rpy([py3d.pi/3,py3d.pi/6,0])
camera_axis @= transform
camera @= transform
py3d.render(camera, camera_axis)


### Project 3d points on xy plane

In [ ]:
import py3d
p=(py3d.Vector3.rand(1000)-0.5).U+py3d.Vector3(x=1,y=1,z=2)
py3d.render(p.as_point())
p.z=0
py3d.render(p.as_point())


### Orthographic projection

In [ ]:
import sympy
l, r, t, b, n, f = sympy.symbols("l r t b n f")
m_scale = sympy.Matrix([
    [2/(r-l), 0, 0, 0],
    [0, 2/(t-b), 0, 0],
    [0, 0, 2/(n-f), 0],
    [0, 0, 0, 1]
])
m_translate = sympy.Matrix([
    [1, 0, 0, -(l+r)/2],
    [0, 1, 0, -(b+t)/2],
    [0, 0, 1, -(f+n)/2],
    [0, 0, 0, 1]
])
o=sympy.simplify(m_scale*m_translate).transpose()
o


In [ ]:
w, h = sympy.symbols("w h")
o=o.subs(l, -w/2).subs(r, w/2).subs(t, h/2).subs(b, -h/2)
o

Use perspective fov and aspect to define a orthographic projection

In [ ]:
fov, aspect, distance = sympy.symbols("fov a d")
o=o.subs(w, aspect * h).subs(h, distance*sympy.tan(fov/2)*2)
o

In [ ]:
x, y, z = sympy.symbols("x y z")
sympy.Matrix([x, y, z, 1]).T @ o

### Perspective projection

In [ ]:
from py3d import Transform, pi, Vector3
projection = Transform.from_perspective(pi/2, 1.2, 0, 1000)
projection


In [ ]:
from numpy import allclose
from py3d import Transform, Vector3
assert (Transform([
    [1., 0., 0.3, 0.],
    [0., 1., 0., 0.],
    [0., 0., 0.1, 0.],
    [-1., 0., 0., 1.]
]) == [
    [1., 0., 0.3, 0.],
    [0., 1., 0., 0.],
    [0., 0., 0.1, 0.],
    [-1., 0., 0., 1.]
]).all()
assert Transform().tile(2, 3).shape == (2, 3, 4, 4)
assert Transform().tile(4, 5).n == (4, 5)
assert ((Vector3(x=1).H @ Transform())[..., 0:3] == Vector3(x=1)).all()
assert (Vector3(x=1) @ Transform() == Vector3(x=1)).all()
p = Vector3([1, 2, 3])
T = Transform.from_translation([1, 2, 3])
R = Transform.from_euler("xyz", [0, 0, 1])
assert allclose(p @ T @ R, p @ (T@R))


[↑Top](#Transform)

[←Home](index.html) 